In [ ]:
import pandas as pd
from pathlib import Path

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.


In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline


In [ ]:
import os
import json
from fastai.vision import *


In [ ]:
tfms = get_transforms(do_flip=True, flip_vert=False, max_rotate=None, max_zoom=1.05, max_lighting=None, max_warp=0,
                      xtra_tfms=[brightness(change=(0.3,0.7),p=0.75), contrast(scale=(0.5,1.0),p=0.75)])


In [ ]:
data_path = "../input"
train_label_file = "train.csv"
train_images_folder = "train_images"
test_label_file = "test.csv"
test_images_folder = "test_images"
image_suffix = ".png"
split_pct = 0.2


In [ ]:
fname = os.path.join(data_path, train_label_file)


In [ ]:
df = pd.read_csv(fname)
df.head(5)


In [ ]:
np.random.seed(42)
src = (ImageList.from_csv(data_path, train_label_file, folder=train_images_folder, suffix=image_suffix)
       .split_by_rand_pct(split_pct)
       .label_from_df())
src


In [ ]:
aptos19_stats = ([0.42, 0.22, 0.075], [0.27, 0.15, 0.081])


In [ ]:
data = (src.transform(tfms, size=224)
        .databunch(bs=64).normalize(aptos19_stats))
data.c, data.classes


In [ ]:
data.show_batch(rows=3, figsize=(8, 8))


In [ ]:
arch = models.densenet121


In [ ]:
kappa = KappaScore()
kappa.weights = "quadratic"


In [ ]:
learn = cnn_learner(data, arch, metrics=[kappa], model_dir="/tmp/model/")


In [ ]:
learn.lr_find()
learn.recorder.plot(suggestion=True)


In [ ]:
lr = 1e-3


In [ ]:
learn.fit_one_cycle(5, slice(lr))


In [ ]:
learn.recorder.plot_losses()


In [ ]:
learn.save('headonly')


In [ ]:
learn.load('headonly');


In [ ]:
learn.unfreeze()


In [ ]:
learn.lr_find()
learn.recorder.plot(suggestion=True)


In [ ]:
class SaveBestModel(Recorder):
    def __init__(self, learn, name):
        super().__init__(learn)
        self.name = name
        self.best_loss = None
        self.save_method = self.save_when_loss
        
    def save_when_loss(self, metrics):
        loss = metrics[0]
        if self.best_loss == None or loss < self.best_loss:
            self.best_loss = loss
            self.learn.save(self.name, return_path=True)
            print("Saved the model for minimum loss {:.6f}".format(self.best_loss))
            
    def on_epoch_end(self, last_metrics=MetricsList,**kwargs:Any):
        self.save_method(last_metrics)


In [ ]:
best_model = SaveBestModel(learn, name = "best_model")


In [ ]:
learn.fit_one_cycle(2, slice(5e-4, lr/5))


In [ ]:
learn.save('best_model')


In [ ]:
learn.load('best_model');


In [ ]:
sample_df = pd.read_csv('../input/sample_submission.csv')
sample_df.head()


In [ ]:
test = ImageList.from_df(sample_df, data_path, folder=test_images_folder, suffix=image_suffix)


In [ ]:
learn.data.add_test(test, label=test_label_file)


In [ ]:
preds = learn.get_preds(ds_type=DatasetType.Test)


In [ ]:
preds1, y = preds


In [ ]:
sample_df.diagnosis = preds1.argmax(1)
sample_df.head(10)


In [ ]:
sample_df.to_csv('submission.csv',index=False)
_ = sample_df.hist()
